# 5. Sequence to Sequence with Attention

<br>

## 강의 소개

- Sequence를 Encoding와 Decoding할 수 있는 **sequence to sequence**에 대해 알아봅니다.
- **Sequence to sequence**는 encoder와 decoder로 이루어져 있는 framework으로 대표적인 자연어 처리 architecture 중 하나입니다. Encoder와 Decoder로는 다양한 알고리즘이 사용될 수 있지만 이번 시간에는 **RNN과 Attention**을 결합한 sequence to sequence 모델을 학습합니다.
- 앞선 강의에서 설명드렸던 것처럼 RNN 모델이 갖고 있는 단점을 보완하고자 **Attention**(논문에서는 alignment로 표현되고 있습니다) 기법이 처음 등장했습니다. 다양한 Attention의 종류와 이를 활용한 translation task에 대해서 알아봅니다

<br>

## Further Reading

- [Sequence to sequence learning with neural networks, ICML’14](https://arxiv.org/abs/1409.3215)
- [Effective Approaches to Attention-based Neural Machine Translation, EMNLP 2015](https://arxiv.org/abs/1508.04025)
- [CS224n(2019)_Lecture8_NMT](https://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture08-nmt.pdf)

<br>

## 5.1 Seq2seq Model

- It takes a <font color='red'>sequence of words as input</font> and gives a <font color='red'>sequence of words as output</font>
- Seq2seq 모델은 RNN 의 구조 중 Many-to-many 에 해당한다.
- 입력 시퀀스를 모두 다 읽은 후 출력 시퀀스를 생성한다.
  - 입력과 출력 시퀀스 모두 단어 단위의 문장이다.

<br>

- It composed of an <font color='red'>**encoder**</font> and a <font color='red'>**decoder**</font>
- 입력 문장을 읽어들이는 RNN 모델을 인코더(encoder)라고 한다.
- 출력 문장을 생성하는 RNN 모델을 디코더(decoder)라고 한다.
- 인코더와 디코더는 서로 파라미터들을 공유하지 않는다.
- 아래 그림에서 RNN 의 세부구조를 살펴보면 RNN 모델로서 LSTM 을 사용한 것을 확인할 수 있다.

<br>

- Seq2seq 모델의 인코더에서 마지막 단어까지 읽어들인 후 마지막 time step에서 나오는 hidden state vector는 디코더에 첫 번째 hidden state vector($h_0$)로 사용된다.
  - 이 $h_0$ 는 인코더에서 수집한 정보들을 잘 담고 있다가 이 정보를 바탕으로 디코더는 순차적으로 대응하는 단어를 출력으로 예측하게 된다.

<br>

- 문장을 생성하는 task 에서 첫 번째 단어로 넣어주는 것을 start token(`<START>`, `<SoS>`) 이라고 한다.
- 이러한 특수 단어를 vocabulary 상에 정의해두고 이를 디코더의 첫 번째 time step 의 입력으로 넣어줌으로서 첫 번째 단어에 대한 예측을 실행한다.
- 디코더에서 문장 생성이 끝나는 시점은 end token(`<END>`, `<EoS>`)이 등장할 때 이다.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=10VdBQbxvhLlViSM7ASg35YXkSBpzg66S' width=800/>

- 출처: Sequence to sequence learning with neural networks, ICML’14

<br>

## 5.2 Recall: Types of RNNs

- Sequence-to-sequence
  - Machine Translation

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=10_HulVJ2hghzy4TbEy8C28AM1EVb83mN' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

## 5.3 Seq2seq Model with Attention

- Attention provides a solution to the bottleneck problem
- Seq2seq 모델에서 추가적인 모듈로서 Attention 을 활용할 수 있다.
- Attention 모듈의 motivation은 Seq2seq 모델에서 사용하는 RNN 기반의 모델 구조가 입력 시퀀스의 길이에 상관없이 **크기가 정해진 hidden state vector**에 필요한 정보를 저장하여 전달되는 특성의 한계를 느꼈기 때문이다.
- Attention 을 사용하게 되면 디코더에서는 인코더의 마지막 time step 에서 나온 hidden state vector 하나에만 의존하는 것이 아니라 인코더의 각각의 time step 에서 나온 hidden state vector들을 사용하게 된다.

<br>

### 5.3.1 Core idea

- At each time step of the decoder, focus on a particular part of the source sequence

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12ptVBI1YcS-zce8BF14pTZDyP2C5T2_e' width=800/>

- 출처: https://google.github.io/seq2seq/

<br>

### 5.3.2 Process

- Attention 모듈이 동작하는 과정을 자세히 살펴보자.
- 프랑스어 문장을 영어로 번역하는 task를 예를 들고 있다.
- RNN 모듈이 인코더에서 수행이 될 때 인코더의 hidden state vector 가 4개의 차원으로 이루어져 있다고 하자.
  - $h_{1}^{(e)}, h_{2}^{(e)}, h_{3}^{(e)}, h_{4}^{(e)}$

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=10g84A30mS4M-niqGlTAah8Qee8MyVlKU' width=400/>

<br>

- 먼저 인코더의 마지막 time step 의 hidden state vector $h_{4}^{(e)}$가 디코더의 $h_0$ 로서 입력으로 들어간다.
- 디코더의 첫 번째 time step 에선 인코더에서 들어온 $h_0$와 `<START>` token으로 주어지는 입력 벡터를 사용하여 hidden state vector $h_{1}^{(d)}$를 생성한다.

<br>

- $h_{1}^{(d)}$를 다음 단어를 예측하기 위해 사용할 뿐만 아니라 이 벡터를 인코더에서 주어진 4개의 hidden state vector 와 각각 내적 연산을 수행하여 **Attention scores** 를 생성한다.
  - 이 Attention scores은 내적에 기반한 유사도로서 볼 수 있다.

<br>

- 이 값들을 softmax 를 통과시켜서 확률값으로 변환시켜주고 이를 **Attention distribution**이라고 한다.
  - 확률값으로 구성된 해당 벡터를 **Attention vector**라고 부른다.
- Use the attention distribution to take a weighted sum of the encoder hidden states
  - 이 확률값들은 인코더의 hidden state vector에 부여되는 가중치로서 사용된다.

<br>

- 인코더의 hidden state vector 에 Attention distribution 확률값들을 곱하여 가중평균을 구하여 하나의 인코딩 벡터를 구할 수 있다.
- 이렇게 구해진 Attention output를 **Context vector** 라고 부른다.
- The attention output mostly contains information the hidden states that received high attention

<br>

- Attention 모듈은 Attention scores 와 Attention distribution 을 구하는 부분으로 정의한다.
- Attention 모듈의 입력
  - decoder hidden state vector 1개
  - encoder hidden state vector 4개
- Attention 모듈의 출력
  - encoder hidden state vector 의 가중평균된 하나의 벡터

<br>

- Concatenate attention output with decoder hidden state, then use to compute $\widehat{y_{1}}$ as before
- 디코더의 hidden state vector 와 Attention module 에 의해 만들어진 context 벡터가 concatenate 되어서 output layer의 입력으로 들어간다.
- 이 output layer 를 통해 다음에 나올 단어를 예측하게 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=10or0PzXfJnwoHR1gVulZdy3x6WI8Ye_d' width=400/>

<br>

- 마찬가지로 디코더의 두 번째 time step에서는 이전 time step에서의 출력 벡터를 입력 벡터와 전달받은 hidden state vector $h_{1}^{(d)}$를 사용하여 새로운 hidden state vector $h_{2}^{(d)}$를 생성한다.
- 이 벡터를 앞에서와 같이 Attention 모듈을 적용하여 새로운 context vector 를 만들어서 다음에 나올 단어를 예측하게 된다.
  - 동일한 Attention 모듈을 사용하되 Attention scores 를 계산할 때 새롭게 생성된 $h_{2}^{(d)}$를 이용한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=10oxJsJaMuF-tT_a7tki5sFKXxOtx-5D9' width=470/>

<br>

- 위와 같은 과정을 3번째 time step 에서도 똑같이 실시한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=10toDI2lGUevPqK5p4ezxQqxpKa5rv5UL' width=550/>

<br>

- 위와 같은 과정을 4번째 time step 에서도 똑같이 실시한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=10uB4N1Lr7IbGP9g9DX436rIokHUfVnMI' width=620/>


<br>

- 위와 같은 과정을 5번째 time step 에서도 똑같이 실시한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=10uaYgqj1rkvhFDZlkdPP3drKn_tRwrDl' width=690/>

<br>

- 위와 같은 과정을 6번째 time step 에서도 똑같이 실시한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=112a_coDDNXmTUincCBpv3D8bB_bU9sfD' width=760/>

<br>

### 5.3.3 Teacher Forcing

- 디코더에서 예측한 단어가 잘못된 단어인 경우 그 단어를 다음 time step의 입력 벡터로 사용하게 되면 계속해서 잘못된 단어를 예측하게 된다.
- 모델 학습 시 디코더에 입력 단어 벡터를 넣어줄 때 제대로된 단어를 강제로 넣어주는 것을 Teacher Forcing 이라고 한다.
- inference 단계에서는 teacher forcing 을 사용하지 않고 디코더에서의 각 time step 에서의 출력 단어 벡터를 다음 time step의 입력 단어 벡터로 사용해야 한다.

<br>

## 5.4 Different Attention Mechanisms

- 위의 예시에서 디코더의 hidden state vector와 인코더의 hidden state vector들과의 유사도(attention scores)를 구할 때 내적을 사용한다고 했는 데, 사실 이 유사도를 구하는 방법에는 여러 가지가 있다.

<br>

### 5.4.1 Luong attention

- they get the decoder hidden state at <font color='red'>time $t$</font>
- Then calculate attention scores
- From that get the context vector which will be concatenated with hidden state of the decoder and then predict the output.

<br>

- Luong has different types of alignments.
  - $h_t$ : 디코더에서 주어지는 hidden state vector
  - $\overline{\boldsymbol{h}}_{s}$ : 인코더의 각 time step 에서의 hidden state vector들

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=11CI1mprkgzO2MF06bXj6VSeQDribLLh3' width=500/>

<br>

- dot product
  - 내적 수행
  - 내적 수행 시 중간에 항등(단위)행렬을 사용하는 것과 동일한 값이 계산된다.
- generalized dot product
  - 내적 수행 시 중간에 항등(단위)행렬 대신 다른 값을 갖는 행렬을 사용
  - 가중치를 부여하는 것으로 생각
- concat (Bahdanau)
  - $h_t$ 와 $\overline{\boldsymbol{h}}_{s}$ 를 이용해서 새로운 학습 가능한 neural network 를 생성
  - **이 벡터들을 concatenate** 하여 fully connected layer 를 통과시켜 하나의 스칼라값을 갖도록 모델을 구성할 수 있다.
  - 중간에 hidden layer를 추가하는 것도 가능하다.
  - 여기서 사용되는 파라미터들의 최적화는 Attention module의 backpropagation 과정에 의해서 업데이트될 수 있다.

<br>

### 5.4.2 Bahdanau attention

- At time $t$, we consider the hidden state of the decoder at <font color='red'>time $t$ − 1</font>.
- Then we calculate the alignment, context vectors as above.
- But then we concatenate this context with hidden state of the decoder at time $t$ − 1.
- So before the softmax, this concatenated vector goes inside a LSTM unit.

<br>

- Bahdanau has only a **concat-score alignment** model.

<br>

## 5.5 Attention is Great!

- Attention significantly improves NMT performance
  - It is useful to allow the decoder to focus on particular parts of the source
- Attention solves the bottleneck problem
  - Attention allows the decoder to look directly at source; bypass the bottleneck
- Attention helps with vanishing gradient problem
  - Provides a shortcut to far-away states
- Attention provides some interpretability(해석가능성)
  - By inspecting attention distribution, we can see what the decoder was focusing on
  - The network just learned alignment by itself

<br>

## 5.6 Attention Examples in Machine Translation

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=11Rh5A-7o260BGye7D0iXuiE88O9BzBD0' width=700/>

- It properly learns grammatical orders of words
- It skips unnecessary  words such as an article

<br>

## 5.7 실습: Seq2seq

1. Encoder 를 구현한다.
2. Decoder 를 구현한다.
3. Seq2seq 모델을 구축하고 사용한다.

<br>

### 5.7.1 필요 패키지 import

In [ ]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch
import random
from pprint import pprint

<br>

### 5.7.2 데이터 전처리

- `src_data` 를 `trg_data` 로 바꾸는 task 를 수행하기 위한 sample data 이다.
- 전체 단어 수는 100개이고 다음과 같이 pad token, start token, end token 의 id 도 정의한다.

In [ ]:
vocab_size = 100
pad_id = 0
sos_id = 1
eos_id = 2

src_data = [
  [3, 77, 56, 26, 3, 55, 12, 36, 31],
  [58, 20, 65, 46, 26, 10, 76, 44],
  [58, 17, 8],
  [59],
  [29, 3, 52, 74, 73, 51, 39, 75, 19],
  [41, 55, 77, 21, 52, 92, 97, 69, 54, 14, 93],
  [39, 47, 96, 68, 55, 16, 90, 45, 89, 84, 19, 22, 32, 99, 5],
  [75, 34, 17, 3, 86, 88],
  [63, 39, 5, 35, 67, 56, 68, 89, 55, 66],
  [12, 40, 69, 39, 49]
]

trg_data = [
  [75, 13, 22, 77, 89, 21, 13, 86, 95],
  [79, 14, 91, 41, 32, 79, 88, 34, 8, 68, 32, 77, 58, 7, 9, 87],
  [85, 8, 50, 30],
  [47, 30],
  [8, 85, 87, 77, 47, 21, 23, 98, 83, 4, 47, 97, 40, 43, 70, 8, 65, 71, 69, 88],
  [32, 37, 31, 77, 38, 93, 45, 74, 47, 54, 31, 18],
  [37, 14, 49, 24, 93, 37, 54, 51, 39, 84],
  [16, 98, 68, 57, 55, 46, 66, 85, 18],
  [20, 70, 14, 6, 58, 90, 30, 17, 91, 18, 90],
  [37, 93, 98, 13, 45, 28, 89, 72, 70]
]

<br>

- 각각의 데이터를 전처리한다.

In [ ]:
trg_data = [[sos_id] + seq + [eos_id] for seq in tqdm(trg_data)]

for t in trg_data:
    print(t)

100%|██████████| 10/10 [00:00<00:00, 14706.54it/s]


[1, 75, 13, 22, 77, 89, 21, 13, 86, 95, 2]
[1, 79, 14, 91, 41, 32, 79, 88, 34, 8, 68, 32, 77, 58, 7, 9, 87, 2]
[1, 85, 8, 50, 30, 2]
[1, 47, 30, 2]
[1, 8, 85, 87, 77, 47, 21, 23, 98, 83, 4, 47, 97, 40, 43, 70, 8, 65, 71, 69, 88, 2]
[1, 32, 37, 31, 77, 38, 93, 45, 74, 47, 54, 31, 18, 2]
[1, 37, 14, 49, 24, 93, 37, 54, 51, 39, 84, 2]
[1, 16, 98, 68, 57, 55, 46, 66, 85, 18, 2]
[1, 20, 70, 14, 6, 58, 90, 30, 17, 91, 18, 90, 2]
[1, 37, 93, 98, 13, 45, 28, 89, 72, 70, 2]


In [ ]:
def padding(data, is_src=True):
    max_len = len(max(data, key=len))
    print(f"Maximum sequence length: {max_len}")

    valid_lens = []
    for i, seq in enumerate(tqdm(data)):
        valid_lens.append(len(seq))
        if len(seq) < max_len:
            data[i] = seq + [pad_id] * (max_len - len(seq))

    return data, valid_lens, max_len

In [ ]:
src_data, src_lens, src_max_len = padding(src_data)
trg_data, trg_lens, trg_max_len = padding(trg_data)

Maximum sequence length: 15


100%|██████████| 10/10 [00:00<00:00, 16339.32it/s]


Maximum sequence length: 22


100%|██████████| 10/10 [00:00<00:00, 72691.58it/s]


In [ ]:
# B: batch size, S_L: source maximum sequence length, T_L: target maximum sequence length
src_batch = torch.LongTensor(src_data)  # (B, S_L)
src_batch_lens = torch.LongTensor(src_lens)  # (B)
trg_batch = torch.LongTensor(trg_data)  # (B, T_L)
trg_batch_lens = torch.LongTensor(trg_lens)  # (B)

print(src_batch.shape)
print(src_batch_lens.shape)
print(trg_batch.shape)
print(trg_batch_lens.shape)

torch.Size([10, 15])
torch.Size([10])
torch.Size([10, 22])
torch.Size([10])


<br>

- PackedSquence를 사용을 위해 source data를 기준으로 정렬합니다.

In [ ]:
src_batch_lens, sorted_idx = src_batch_lens.sort(descending=True)
src_batch = src_batch[sorted_idx]
trg_batch = trg_batch[sorted_idx]
trg_batch_lens = trg_batch_lens[sorted_idx]

print(src_batch)
print(src_batch_lens)
print(trg_batch)
print(trg_batch_lens)

tensor([[39, 47, 96, 68, 55, 16, 90, 45, 89, 84, 19, 22, 32, 99,  5],
        [41, 55, 77, 21, 52, 92, 97, 69, 54, 14, 93,  0,  0,  0,  0],
        [63, 39,  5, 35, 67, 56, 68, 89, 55, 66,  0,  0,  0,  0,  0],
        [ 3, 77, 56, 26,  3, 55, 12, 36, 31,  0,  0,  0,  0,  0,  0],
        [29,  3, 52, 74, 73, 51, 39, 75, 19,  0,  0,  0,  0,  0,  0],
        [58, 20, 65, 46, 26, 10, 76, 44,  0,  0,  0,  0,  0,  0,  0],
        [75, 34, 17,  3, 86, 88,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [12, 40, 69, 39, 49,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [58, 17,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [59,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
tensor([15, 11, 10,  9,  9,  8,  6,  5,  3,  1])
tensor([[ 1, 37, 14, 49, 24, 93, 37, 54, 51, 39, 84,  2,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [ 1, 32, 37, 31, 77, 38, 93, 45, 74, 47, 54, 31, 18,  2,  0,  0,  0,  0,
          0,  0,  0,  0],
        [ 1, 20, 70, 14,  6, 58, 90,

<br>

### 5.7.3 Encoder 구현

In [ ]:
embedding_size = 256
hidden_size = 512
num_layers = 2
num_dirs = 2
dropout = 0.1

<br>

- Bidirectional GRU를 이용한 Encoder입니다.
  - `self.embedding`: word embedding layer.
  - `self.gru`: encoder 역할을 하는 Bi-GRU.
  - `self.linear`: 양/단방향 concat된 hidden state를 decoder의 hidden size에 맞게 linear transformation.


In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru = nn.GRU(
            input_size=embedding_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True if num_dirs > 1 else False,
            dropout=dropout
        )
        self.linear = nn.Linear(num_dirs * hidden_size, hidden_size)

    def forward(self, batch, batch_lens): # batch: (B, S_L), batch_lens: (B)
        # d_w: word embedding size
        batch_emb = self.embedding(batch) # (B, S_L, d_w)
        batch_emb = batch_emb.transpose(0, 1) # (S_L, B, d_w)

        packed_input = pack_padded_sequence(batch_emb, batch_lens)

        h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size)) # (num_layers*num_dirs, B, d_h) = (4, B, d_h)
        packed_outputs, h_n = self.gru(packed_input, h_0) # h_n: (4, B, d_h)
        outputs = pad_packed_sequence(packed_outputs)[0] # outputs: (S_L, B, 2d_h)

        forward_hidden = h_n[-2, :, :]
        backward_hidden = h_n[-1, :, :]  
        hidden = self.linear(torch.cat((forward_hidden, backward_hidden), dim=-1)).unsqueeze(0) # (1, B, d_h)

        return outputs, hidden

In [ ]:
encoder = Encoder()

<br>

### 5.7.4 Decoder 구현

- 동일한 설정의 Bi-GRU로 만든 Decoder입니다.
  - `self.embedding`: word embedding layer.
  - `self.gru`: decoder 역할을 하는 Bi-GRU.
  - `self.output_layer`: decoder에서 나온 hidden state를 `vocab_size`로 linear transformation하는 layer.

In [ ]:
class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_size)
    self.gru = nn.GRU(
        input_size=embedding_size, 
        hidden_size=hidden_size,
    )
    self.output_layer = nn.Linear(hidden_size, vocab_size)

  def forward(self, batch, hidden):  # batch: (B), hidden: (1, B, d_h)
    batch_emb = self.embedding(batch)  # (B, d_w)
    batch_emb = batch_emb.unsqueeze(0)  # (1, B, d_w)

    outputs, hidden = self.gru(batch_emb, hidden)  # outputs: (1, B, d_h), hidden: (1, B, d_h)
    
    # V: vocab size
    outputs = self.output_layer(outputs)  # (1, B, V)

    return outputs.squeeze(0), hidden

In [ ]:
decoder = Decoder()

<br>

### 5.7.5 Seq2seq 모델 구축

- 생성한 encoder와 decoder를 합쳐 Seq2seq 모델을 구축합니다.
  - `self.encoder`: encoder.
  - `self.decoder`: decoder.

In [ ]:
class Seq2seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src_batch, src_batch_lens, trg_batch, teacher_forcing_prob=0.5):
        # src_batch: (B, S_L)
        # src_batch_lens: (B)
        # trg_batch: (B, T_L)

        _, hidden = self.encoder(src_batch, src_batch_lens) # hidden: (1, B, d_h)

        input_ids = trg_batch[:, 0] # (B)
        batch_size = src_batch.shape[0]
        outputs = torch.zeros(trg_max_len, batch_size, vocab_size) # (T_L, B, V)

        for t in range(1, trg_max_len):
            decoder_outputs, hidden = self.decoder(input_ids, hidden) # decoder_outputs: (B, V), hidden: (1, B, d_h)

            outputs[t] = decoder_outputs
            _, top_ids = torch.max(decoder_outputs, dim=-1) # top_ids: (B)

            input_ids = trg_batch[:, t] if random.random() > teacher_forcing_prob else top_ids

        return outputs

In [ ]:
seq2seq = Seq2seq(encoder, decoder)

<br>

### 5.7.6 모델 사용해보기

- 학습 과정이라고 가정하고 모델에 input을 넣어봅니다.

In [ ]:
outputs = seq2seq(src_batch, src_batch_lens, trg_batch)

print(outputs)
print(outputs.shape)

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.2533, -0.1049, -0.0585,  ..., -0.0565,  0.1070, -0.0453],
         [ 0.2580, -0.1068, -0.0577,  ...,  0.0177,  0.1002, -0.0476],
         [ 0.2205, -0.1333, -0.0125,  ...,  0.0085,  0.0785, -0.0423],
         ...,
         [ 0.1932, -0.1302, -0.0284,  ...,  0.0264,  0.0803, -0.0160],
         [ 0.2197, -0.1176, -0.0521,  ...,  0.0193,  0.0980, -0.0733],
         [ 0.2213, -0.1205, -0.0525,  ...,  0.0288,  0.1000, -0.0395]],

        [[ 0.1053,  0.0687, -0.0634,  ...,  0.0505, -0.0122,  0.1181],
         [ 0.0684,  0.0842, -0.0570,  ..., -0

<br>

- Language Modeling 에 대한 loss 계산을 위해 shift 한 target 과 비교한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=11UNRhJug0bK6VVfhm5Nj5fi8Y7h3v6p8' width=800/>

In [ ]:
loss_function = nn.CrossEntropyLoss()

preds = outputs[1:, :, :].transpose(0, 1) # (B, T_L-1, V)
loss = loss_function(
    preds.contiguous().view(-1, vocab_size),
    trg_batch[:, 1:].contiguous().view(-1, 1).squeeze(1)
)

print(loss)

tensor(4.6455, grad_fn=<NllLossBackward>)


<br>

- 실제 inference에선 teacher forcing 없이 이전 결과만을 가지고 생성합니다.

In [ ]:
src_sent = [4, 10, 88, 46, 72, 34, 14, 51]
src_len = len(src_sent)

src_batch = torch.LongTensor(src_sent).unsqueeze(0)  # (1, L)
src_batch_lens = torch.LongTensor([src_len])  # (1)

_, hidden = seq2seq.encoder(src_batch, src_batch_lens)  # hidden: (1, 1, d_h)

In [ ]:
input_id = torch.LongTensor([sos_id]) # (1)
output = []

for t in range(1, trg_max_len):
  decoder_output, hidden = seq2seq.decoder(input_id, hidden)  # decoder_output: (1, V), hidden: (1, 1, d_h)

  _, top_id = torch.max(decoder_output, dim=-1)  # top_ids: (1)

  if top_id == eos_id:
    break
  else:
    output += top_id.tolist()
    input_id = top_id

In [ ]:
print(output)

[77, 5, 8, 33, 36, 77, 6, 65, 58, 31, 58, 31, 58, 31, 58, 31, 58, 31, 58, 31, 58]
